In [1]:
import math
import pandas as pd
from lxml import etree
from frccolors import RoboColor
import tbapy


tba = tbapy.TBA('LenYI5DMTy1bQIoP4ralqFVd5g5JvTY9YBEBAubPktLTelEdPukJZ1RVLIV1Ypfu')
not_found_string = "N/A"
imperialism_loc = "/home/will/Documents/Projects/frc_counties/data/imperialism_map_start.csv"
svg_loc = "/home/will/Documents/Projects/frc_counties/data/counties.svg"
custom_colors_loc = "/home/will/Documents/Projects/frc_counties/data/team_colors.csv"
target_loc = "frc_imperialism_map.svg"
year = 2019
end_date = '2019-03-27'
# choose from: Regional, Championship Division, Offseason, Preseason, District Championship, Championship Finals, District, District Championship Division
acceptable_event_types = ["Regional", "Championship Division", "District Championship", "Championship Finals", "District", "District Championship Division"]#, "Offseason"]
champ_types = ["District Championship", "Championship Finals"]
roboColor = RoboColor(custom_colors_loc)

In [2]:
# load imperialism start into dataframe
imp = pd.read_csv(imperialism_loc)
imp.head(6)

,County,Teams
0,c39167,7515
1,c49007,6844
2,c46013,3300
3,c51710,1793
4,c29107,2457
5,c55009,"1716,6318"


In [3]:
# , and then into custom datastructure
county_assignment = {}
for index, row in imp.iterrows():
    county = row["County"]
    teams = row["Teams"].split(",")
    for i in range(len(teams)):
        teams[i] = int(teams[i])
        if county in county_assignment:
            county_assignment[county].append(teams[i])
        else:
            county_assignment[county] = [teams[i]]

In [12]:
# helper functions for svg
def writeNewSVGWithColorDictionary(color_dict, tooltips_dict = None):
    tree = etree.parse(svg_loc)
    root = tree.getroot()
    paths = root[3]
    for path in paths:
        path_county_id = path.attrib["id"]
        if path_county_id not in color_dict:
            continue
        if color_dict[path_county_id] == "CONTESTED": #fill="url(#contested)"
            path.attrib["fill"] = "url(#contested);"
        else:
            path.attrib["style"] = "fill: " + color_dict[path_county_id] + ";"
            
        if tooltips_dict != None and path_county_id in tooltips_dict:
            path[0].text += " - " + str(tooltips_dict[path_county_id])
    etree.ElementTree(root).write(target_loc, pretty_print=True)

# helper functions to assist modifying datastructure based on events that happen
def extract_enddate(event): # used for sorting
    return event.end_date

def extract_champtype(event):  # used for sorting
    return event.event_type_string in champ_types

def landClaim(winningTeam, list_of_losers):
    for county in county_assignment:
        countyOwners = county_assignment[county]
        foundLoser = False
        for loser in list_of_losers:
            if loser in countyOwners:
                county_assignment[county].remove(loser)
                foundLoser = True
        # if the owner won land, add them to the county if they're not already in it
        if foundLoser and winningTeam not in county_assignment[county]: 
            county_assignment[county].append(winningTeam)

In [5]:
# load in all events & filter
init_events = tba.events(year=year)
init_events.sort(key=extract_champtype)
init_events.sort(key=extract_enddate)
events = []
for event in init_events:
    if event.event_type_string in acceptable_event_types and event.end_date <= end_date:
        events.append(event)
print(str(len(events)) + " events loaded.")

115 events loaded.


In [6]:
# go through each event, grab winner, generate list of every team at event not on winning alliance, call helper
# function

numEvent = 1
for event in events:
    event_alliances = tba.event_alliances(event.key)
    event_teams = tba.event_teams(event.key, keys=True)
    for i in range(len(event_teams)):
        event_teams[i] = int(event_teams[i][3:])
    winner = None
    partners = []
    list_of_losers = []
    for alliance in event_alliances:
        if (event.event_type_string not in champ_types and alliance.status["status"] == 'won') or (event.event_type_string in champ_types and alliance.status["status"] == 'won' and alliance.status["level"] == 'f'):
            converted_all = []
            for i in range(len(alliance.picks)):
                converted_all.append(int(alliance.picks[i][3:]))
            winner = converted_all[0]
            partners = []
            for i in range(1, len(converted_all)):
                partners.append(converted_all[i])
            break
    if event.event_type_string not in champ_types or event.playoff_type == None or event.playoff_type == 0:
        for event_team in event_teams:
            if event_team != winner and event_team not in partners:
                list_of_losers.append(event_team)
    else: # grab championship finals event alliances and mark losers
        for alliance in event_alliances:
            for i in range(len(alliance.picks)):
                alliance.picks[i] = int(alliance.picks[i][3:])
            if winner in alliance.picks:
                continue
            else:
                for op_loser in alliance.picks:
                    list_of_losers.append(op_loser)
    # call helper func
    landClaim(winner, list_of_losers)
    if numEvent <= 5 or numEvent % 10 == 0:
        print(str(numEvent) + "/" + str(len(events)) + " loaded so far.")
    numEvent += 1

1/115 loaded so far.
2/115 loaded so far.
3/115 loaded so far.
4/115 loaded so far.
5/115 loaded so far.
10/115 loaded so far.
20/115 loaded so far.
30/115 loaded so far.
40/115 loaded so far.
50/115 loaded so far.
60/115 loaded so far.
70/115 loaded so far.
80/115 loaded so far.
90/115 loaded so far.
100/115 loaded so far.
110/115 loaded so far.


In [13]:
# create color dictionary for displaying data
color_dict = {}
hover_dict = {}
for county_fips_key in county_assignment:
    if len(county_assignment[county_fips_key]) > 1:
        color_dict[county_fips_key] = "CONTESTED"
        toAdd = str(county_assignment[county_fips_key][0])
        for i in range(1, len(county_assignment[county_fips_key])):
            toAdd += ", " + str(county_assignment[county_fips_key][i])
        hover_dict[county_fips_key] = toAdd
    else:
        color_dict[county_fips_key] = roboColor.getTeamRGB(county_assignment[county_fips_key][0])
        hover_dict[county_fips_key] = str(county_assignment[county_fips_key][0])

In [15]:
# write to file
writeNewSVGWithColorDictionary(color_dict, hover_dict)

In [9]:
# sort and print teams w/ most land
team_county_counts = {}
for county_fips in county_assignment:
    for team in county_assignment[county_fips]:
        if team not in team_county_counts:
            team_county_counts[team] = 1
        else:
            team_county_counts[team] += 1
largest_owners = sorted(team_county_counts, key=team_county_counts.get, reverse=True)
rank = 1
for team in largest_owners:
    print(str(rank) + ". " + str(team) + " - " + str(team_county_counts[team]) + " counties")
    rank += 1

1. 364 - 275 counties
2. 1619 - 253 counties
3. 4451 - 207 counties
4. 1481 - 167 counties
5. 5172 - 138 counties
6. 118 - 136 counties
7. 5013 - 132 counties
8. 832 - 123 counties
9. 935 - 123 counties
10. 2052 - 122 counties
11. 48 - 107 counties
12. 7457 - 102 counties
13. 148 - 89 counties
14. 2468 - 88 counties
15. 4020 - 88 counties
16. 1414 - 81 counties
17. 3750 - 79 counties
18. 2910 - 76 counties
19. 612 - 75 counties
20. 346 - 69 counties
21. 1501 - 69 counties
22. 1323 - 67 counties
23. 386 - 61 counties
24. 5801 - 59 counties
25. 6377 - 49 counties
26. 5511 - 46 counties
27. 5404 - 45 counties
28. 2059 - 44 counties
29. 1599 - 44 counties
30. 1756 - 43 counties
31. 20 - 40 counties
32. 78 - 40 counties
33. 948 - 38 counties
34. 870 - 36 counties
35. 5927 - 36 counties
36. 4414 - 33 counties
37. 6400 - 26 counties
38. 1519 - 26 counties
39. 4004 - 25 counties
40. 6569 - 23 counties
41. 2075 - 22 counties
42. 56 - 21 counties
43. 3844 - 20 counties
44. 2791 - 19 counties
45.